In [1]:
import pandas as pd
import json
import requests
import datetime as datetime
from datetime import date
from dateutil.parser import *

In [2]:
url = "https://services.arcgis.com/afSMGVsC7QlRK1kZ/arcgis/rest/services/Police_Use_of_Force/FeatureServer/0/query?where=1%3D1&outFields=*&outSR=4326&f=json"

In [3]:
response = requests.get(url)

In [4]:
response_json = response.json()

In [6]:
response_json.keys()

dict_keys(['objectIdFieldName', 'uniqueIdField', 'globalIdFieldName', 'geometryType', 'spatialReference', 'fields', 'features'])

In [45]:
[response_json["features"][x]['attributes']['ResponseDate'] for x in range(len(response_json["features"])) if type(response_json["features"][x]['attributes']['ResponseDate']) != int]

[None, None, None, None, None, None, None]

In [42]:
response_json["features"][30]['attributes'].keys()

dict_keys(['PoliceUseOfForceID', 'CaseNumber', 'ResponseDate', 'Problem', 'Is911Call', 'PrimaryOffense', 'SubjectInjury', 'ForceReportNumber', 'SubjectRole', 'SubjectRoleNumber', 'ForceType', 'ForceTypeAction', 'Race', 'Sex', 'EventAge', 'TypeOfResistance', 'Precinct', 'Neighborhood', 'TotalCityCallsForYear', 'TotalPrecinctCallsForYear', 'TotalNeighborhoodCallsForYear', 'CenterGBSID', 'CenterLatitude', 'CenterLongitude', 'CenterX', 'CenterY', 'DateAdded', 'OBJECTID'])

In [5]:
new_dict = {}
new_dict["Year"] = []
new_dict["YearMonth"] = []
new_dict["Hour"] = []
# new_dict["geometry"] = []
for field in response_json["features"][0]['attributes'].keys():
    new_dict[field] = []
for index in range(len(response_json["features"])):
    a = response_json["features"][index]['attributes']
    b = response_json["features"][index]['geometry']
    
    if a['ResponseDate'] != None:
    
        for key in response_json["features"][index]['attributes']:
        
        
        
            if key == 'ResponseDate':
                    d = datetime.datetime.fromtimestamp(a[key]/1000)
                    e = d.strftime('%Y-%m-%d')
                    # e = d.strftime('%Y-%m-%d %H:%M:%S')
                    new_dict[key].append(e)
                    new_dict["Year"].append(d.strftime('%Y'))
                    new_dict["YearMonth"].append(d.strftime('%Y-%m'))
                    new_dict["Hour"].append(d.strftime('%H'))
            else:
                new_dict[key].append(a[key])
                # new_dict["geometry"].append(b)
    
    
    else:
        pass


    
df = pd.DataFrame(new_dict)

In [66]:
type(new_dict['Year'][0])

str

In [16]:
# Map Data Table - CaseNumber, Lat, Long, Neighborhood, Precinct
# Incident Data Table - everything else

In [6]:
import sqlite3

In [7]:
sqlite3.connect('MNPDUseofForce.sqlite')

In [69]:
cursor = connection.cursor()

In [8]:
from sqlalchemy import create_engine
engine = create_engine('sqlite:///MNPDUseofForce.sqlite')

In [9]:
df.to_sql('MNPD_tbl', con=engine, if_exists='replace', index=False)

In [73]:
conn = engine.connect()
results = pd.read_sql("SELECT Year FROM MNPD_tbl", conn)

In [76]:
results.count()

Year    33774
dtype: int64

In [77]:
engine = create_engine("sqlite:///MNPDUseofForce.sqlite")
data = engine.execute("SELECT Problem FROM MNPD_tbl WHERE Year = 2010")

In [78]:
list(data)

[('Person with a Gun ',),
 ('Unwanted Person ',),
 ('Suspicious Person ',),
 ('Fight ',),
 ('Disturbance ',),
 ('Fight ',),
 ('Suspicious Person ',),
 ('Domestic Abuse-In Progress ',),
 ('Attempt Pick-Up ',),
 ('Suspicious Person ',),
 ('Disturbance ',),
 ('Disturbance ',),
 ('Traffic Law Enforcement ',),
 ('Disturbance ',),
 ('Loud Party ',),
 ('Unwanted Person ',),
 ('Unknown Wireless/Cell Phone',),
 ('Traffic Law Enforcement ',),
 ('On Site ',),
 ('Check the Welfare ',),
 ('Check the Welfare ',),
 ('Traffic Law Enforcement ',),
 ('Officer Needs Help ',),
 ('Fight ',),
 ('Suspicious Person ',),
 ('Motor Vehicle Chase ',),
 ('Fight ',),
 ('Unwanted Person ',),
 ('Unwanted Person ',),
 ('Fight ',),
 ('Fight ',),
 ('Fight ',),
 ('Unknown Trouble ',),
 ('Fight ',),
 ('Loud Party ',),
 ('Disturbance ',),
 ('Poss Personal Injury Acc ',),
 ('Crim Sex Conduct ',),
 ('Disturbance ',),
 ('Disturbance ',),
 ('Miscellaneous ',),
 ('Suspicious Person ',),
 ('Suspicious Person ',),
 ('Suspicious P

In [11]:
df.keys()

Index(['Year', 'YearMonth', 'Hour', 'PoliceUseOfForceID', 'CaseNumber',
       'ResponseDate', 'Problem', 'Is911Call', 'PrimaryOffense',
       'SubjectInjury', 'ForceReportNumber', 'SubjectRole',
       'SubjectRoleNumber', 'ForceType', 'ForceTypeAction', 'Race', 'Sex',
       'EventAge', 'TypeOfResistance', 'Precinct', 'Neighborhood',
       'TotalCityCallsForYear', 'TotalPrecinctCallsForYear',
       'TotalNeighborhoodCallsForYear', 'CenterGBSID', 'CenterLatitude',
       'CenterLongitude', 'CenterX', 'CenterY', 'DateAdded', 'OBJECTID'],
      dtype='object')